# Importações

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import joblib
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

# Carregar e pré-processar o dataset

In [2]:
# Inicializar o stemmer
stemmer = PorterStemmer()

# Função para aplicar stemming em cada palavra da mensagem
def apply_stemming(text):
    # Verifica se o valor é string, caso contrário retorna texto vazio
    if isinstance(text, str):
        words = word_tokenize(text.lower())  # Tokeniza o texto
        stemmed_words = [stemmer.stem(word) for word in words]  # Aplica stemming em cada palavra
        return ' '.join(stemmed_words)
    else:
        print("erro")
        return ''  # Retorna string vazia se não for texto


In [3]:
# Carregar o dataset
df = pd.read_csv('enron_spam_data.csv', sep=',')

In [4]:
# Substituir valores nulos por strings vazias
df['Message'] = df['Message'].fillna('')

In [ ]:
# Aplicar stemming nas mensagens
df['Message'] = df['Message'].apply(apply_stemming)

In [7]:
# Salvar arquivo pós-processamento
# df.to_csv('datasetPronto.csv', index=False)

In [ ]:
print(df['Message'])

In [7]:
# Separar as features e os rótulos
X = df['Message']
y = df['Spam/Ham'].apply(lambda x: 1 if x == 'spam' else 0)

In [8]:
# Dividir o dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Transformar o texto em vetores de TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Treinar o modelo SVM com kernel RBF


In [10]:
# Inicializar o modelo SVM com kernel RBF
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')  # C é o parâmetro de regularização

In [ ]:
# Treinar o modelo
svm_model.fit(X_train_tfidf, y_train)

In [ ]:
# Salvar o modelo em um arquivo
joblib.dump(svm_model, 'modelo_sem_ajuste_sem_kfold-c1-gSC.pkl')
joblib.dump(vectorizer, 'vectorizer_sem_ajuste_sem_kfold-c1-gSC.pkl')

In [ ]:
#Carregar modelo
svm_model = joblib.load('modelo_sem_ajuste_sem_kfold-c1-gSC.pkl')
vectorizer = joblib.load('vectorizer_sem_ajuste_sem_kfold-c1-gSC.pkl')

In [27]:
# Fazer previsões no conjunto de teste
y_pred = svm_model.predict(X_test_tfidf)

In [25]:
testtxt = ""
test = apply_stemming(testtxt)
testl = vectorizer.transform([test])  # Note a mudança para uma lista
y_pred = svm_model.predict(testl)

In [ ]:
print(y_pred)

In [ ]:
# Avaliar o modelo
print(f"Acurácia: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=['ham', 'spam']))

In [29]:
# Calcular a matriz de confusão
cm = confusion_matrix(y_test, y_pred, labels=[0, 1])

In [30]:
# Criar um objeto de exibição da matriz de confusão
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['ham', 'spam'])

In [ ]:
# Plotar a matriz de confusão
disp.plot(cmap=plt.cm.Blues)
plt.title('Matriz de Confusão')
plt.show()

## Ajustar os Hiperparâmetros

In [21]:
# Definir os parâmetros a serem testados
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],
    'kernel': ['rbf']
}

In [ ]:
# Usar o GridSearchCV para encontrar os melhores parâmetros
grid_search = GridSearchCV(SVC(), param_grid, cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train)

In [ ]:
# Ver os melhores parâmetros
print(f"Melhores parâmetros: {grid_search.best_params_}")

In [ ]:
# Treinar o modelo com os melhores parâmetros
best_svm_model = grid_search.best_estimator_
best_svm_model.fit(X_train_tfidf, y_train)

In [ ]:
# Salvar o modelo otimizado em um arquivo
joblib.dump(best_svm_model, 'modelo_com_ajuste_e_kfold.pkl')
joblib.dump(vectorizer, 'vectorizer_com_ajuste_e_kfold.pkl')

In [ ]:
# best_svm_model = joblib.load('modelo_treinado.pkl')

In [ ]:
# Avaliar o modelo otimizado
y_pred_optimized = best_svm_model.predict(X_test_tfidf)
print(f"Acurácia: {accuracy_score(y_test, y_pred_optimized) * 100:.2f}%")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_optimized, target_names=['ham', 'spam']))

In [ ]:
# Calcular a matriz de confusão
cm = confusion_matrix(y_test, y_pred_optimized, labels=['ham', 'spam'])# Matriz de 

In [ ]:
# Criar um objeto de exibição da matriz de confusão
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['ham', 'spam'])

In [ ]:
# Plotar a matriz de confusão
disp.plot(cmap=plt.cm.Blues)
plt.title('Matriz de Confusão')
plt.show()